<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/Homework2025/2025/HW/JDRath/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extending Code

In [7]:
# import libraries
import numpy as np
import scipy.linalg as la


# Let's calculate the frequencies numerically:

def calculate_eigenfrequencies_3mass(k, m):
    """ calculate the eigenfrequencies for our three mass problem.

    Keyword arguments:
    k -- spring constant (N/m)
    m -- mass (kg)

    returns:
    w1 -- first eigenfrequency
    w2 -- second eigenfrequency
    w3 -- third eigenfrequency
    """
    w0_squared = k / m
    w1 = np.sqrt(2*w0_squared)
    w2 = np.sqrt((2-np.sqrt(2)) * w0_squared)
    w3 = np.sqrt((2+np.sqrt(2)) * w0_squared)
    return w1, w2, w3

# Example values
k = 10  # N/m
m = 1   # kg

w1, w2, w3 = calculate_eigenfrequencies_3mass(k, m)

print("Eigenfrequencies:")
print("ω₁ = {:.4f} rad/s".format(w1))
print("ω₂ = {:.4f} rad/s".format(w2))
print("ω₃ = {:.4f} rad/s".format(w3))

Eigenfrequencies:
ω₁ = 4.4721 rad/s
ω₂ = 2.4203 rad/s
ω₃ = 5.8431 rad/s


In [8]:
def setup_mass_matrix(m_list):
  """
  This function creates you mass matrix using the provided mass list for the diagonal terms.
  This is for a spring system with walls.

  Arguments:
  m_list -- list of masses

  Returns:
  M -- mass matrix
  """
  M = np.diag(m_list)
  return M

In [9]:
M = setup_mass_matrix([1, 1, 1])
M

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [15]:
def setup_stiffness_matrix(k_list, walls=True):
  """
  This function creates you stiffness matrix using the provided spring constant list.
  This is for a spring system with walls.

  Arguments:
  k_list -- list of spring constants
  walls -- boolean, whether or not connected to include walls

  Returns:
  K -- stiffness matrix
  """

  K = np.zeros((len(k_list)-1, len(k_list)-1))
  if walls:

    for i in range(len(k_list)-1):
      K[i, i] = k_list[i] + k_list[i+1] # Diagonal
      if i < len(k_list)-2:
        K[i, i+1] = -k_list[i+1] # Upper Diagonal
        K[i+1, i] = -k_list[i+1] # Lower Diagonal

  else:
    for i in range(len(k_list)-1):
            if i == 0:  # Left-most mass
                K[i, i] = k_list[i+1]
            elif i == len(k_list) - 2:  # Right-most mass
                K[i, i] = k_list[i]
            else:  # Inside masses
                K[i, i] = k_list[i] + k_list[i+1]
            if i < len(k_list) - 2:
                K[i, i+1] = -k_list[i+1]  # Upper diagonal
                K[i+1, i] = -k_list[i+1]  # Lower diagonal

  return K

In [16]:
K = setup_stiffness_matrix([10, 3, 7, 10])
K

array([[13., -3.,  0.],
       [-3., 10., -7.],
       [ 0., -7., 17.]])

# AI creativity

In [ ]:
import numpy as np
from scipy.linalg import eigh

# Define system parameters
Nx, Ny = 3, 3  # Dimensions of the 2D lattice
N = Nx * Ny  # Total number of atoms
masses = np.array([1.0 if (i + j) % 2 == 0 else 2.0 for i in range(Nx) for j in range(Ny)])  # Varying atomic masses
k1_base = 1.0  # Nearest-neighbor spring constant base value
k2_base = 0.5  # Next-nearest-neighbor coupling base value

def decay_function(d):
    return k2_base / d**2  # Decaying interaction strength

def anharmonic_term(x):
    return 0.1 * x**3  # Simple anharmonic term

# Construct the dynamical matrix D
D = np.zeros((N, N))
for i in range(N):
    xi, yi = divmod(i, Ny)
    for dx, dy, k_base in [(1, 0, k1_base), (0, 1, k1_base), (1, 1, decay_function(np.sqrt(2))), (1, -1, decay_function(np.sqrt(2)))]:
        xj, yj = xi + dx, yi + dy
        if 0 <= xj < Nx and 0 <= yj < Ny:
            j = xj * Ny + yj
            k = k_base * (1.0 + anharmonic_term(1.0))  # Including anharmonic term
            D[i, i] += k / masses[i]
            D[i, j] -= k / masses[i]
            D[j, j] += k / masses[j]
            D[j, i] -= k / masses[j]

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = eigh(D)
frequencies = np.sqrt(np.abs(eigenvalues))  # In rad/s

# Print eigenfrequencies and explain the meaning of modes
print("Eigenfrequencies (rad/s):")
for i, freq in enumerate(frequencies):
    print(f"Mode {i+1}: {freq:.4f} rad/s")

print("\nInterpretation of Modes:")
print("Each mode represents a characteristic vibration pattern of the atoms.")
print("- Lower-frequency modes correspond to slow, large-wavelength oscillations.")
print("- Higher-frequency modes involve rapid, small-wavelength oscillations.")
print("- These modes describe how the atoms in the lattice move collectively.")
print("- In real crystals, these are associated with phonons, which affect thermal and acoustic properties.")

# Display eigenvectors to visualize mode shapes
print("\nMode Shapes (Eigenvectors):")
for i in range(N):
    print(f"Mode {i+1}:")
    print(eigenvectors[:, i])
    print()

Eigenfrequencies (rad/s):
Mode 1: 0.3988 rad/s
Mode 2: 0.9165 rad/s
Mode 3: 0.9763 rad/s
Mode 4: 1.4744 rad/s
Mode 5: 1.5730 rad/s
Mode 6: 1.8341 rad/s
Mode 7: 1.8636 rad/s
Mode 8: 1.9509 rad/s
Mode 9: 2.4850 rad/s

Interpretation of Modes:
Each mode represents a characteristic vibration pattern of the atoms.
- Lower-frequency modes correspond to slow, large-wavelength oscillations.
- Higher-frequency modes involve rapid, small-wavelength oscillations.
- These modes describe how the atoms in the lattice move collectively.
- In real crystals, these are associated with phonons, which affect thermal and acoustic properties.

Mode Shapes (Eigenvectors):
Mode 1:
[-0.1953205  -0.39719233 -0.27527375 -0.39719233 -0.28205955 -0.38290621
 -0.27527375 -0.38290621 -0.34926094]

Mode 2:
[-0.27995597 -0.39183189 -0.14899537 -0.39183189 -0.09723813  0.38933358
 -0.14899537  0.38933358  0.50748835]

Mode 3:
[ 7.21860932e-17  4.74619870e-01  4.43820313e-01 -4.74619870e-01
  0.00000000e+00  2.78853919e

This model didn't take too long to construct. Firstly, I used ChatGPT as the AI model and it took roughly 15 correction to finish. I thought this was relatively quick and easy to adapt the physics compared to how long I thought it would take. This model reflects atom vibrations. This can't be observed by the naked eye, but is everywhere around us. In the code it gives an example of crystal lattice. ChatGPT came up with the ideas and I narrowed it down to atom vibrations quite quickly, but it took a while to add the fine details such as decay and the structure I envisioned.